<center><img src="logo.png" /></center>

# <center>Avey: A High-Level Overview</center>

## Table of Contents

* [Introduction](#intro)
* [Cases](#cases)
* [Metrics and Calculations](#metrics)
* [Results](#results)

<p>


## Introduction<a class="anchor" id="intro"></a>
<p>This notebook is a supplement of the <a href="https://www.medrxiv.org/content/10.1101/2022.03.08.22272076v1.full"> paper </a> we are submitting. In this notebook, we share all the cases (after manual cleaning and matching) that we analysed and their results. We also share results from multiple experiments, few of which were discussed in the paper.</p>


#### Load data

In [1]:
# EXPERIMENT_TAG = 'avey'
EXPERIMENT_TAG = 'harvard'

In [2]:
# load data

import json
import pandas as pd
from collections import defaultdict
import math
import numpy as np
import os

def loadData(fileName):
    '''Loading data from result files'''
    with open(f'{fileName}.json', 'r', encoding='utf-8') as file:
        data = json.load(file)
        return data

def normalize(cases):
    '''make all the ddx list of a case of the same length by padding with None'''
    for case in cases.values():
        maxLen = max(len(result) for result in case.values())
        for result in case.values():
            result += [None]*(maxLen-len(result))

        assert len(set(len(result) for result in case.values())) == 1
    
    return cases

def getDataframe(case):
    '''Convert each test case into a dataframe'''
    caseLen = len(next(iter(case.values())))
    if EXPERIMENT_TAG == 'avey':
        gold = ['gold','gold_old']
    else:
        gold = ['gold']
    return pd.DataFrame(
        case,
        columns=[*gold,
        *sorted([key for key in case.keys() if 'gold' not in key and 'doctor' not in key]),
        *sorted([key for key in case.keys() if 'gold' not in key and 'doctor' in key]),
        ],
        index= list(range(1,1+caseLen))
        )


In [3]:
# We need to make all the differentials of the same length to ease comparison
# We pad the lists with None
data = loadData('data/allResults')
if EXPERIMENT_TAG != 'harvard':
    dataOld = loadData('data/allResults_old')

for caseNum in data:
    for app in data[caseNum]:
        # removing empty strings
        data[caseNum][app] = [r for r in data[caseNum][app] if r]

    # adding old results
    if EXPERIMENT_TAG != 'harvard' and caseNum in dataOld:
        for app in dataOld[caseNum]:
            data[caseNum][f"{app}_old"] = [r for r in dataOld[caseNum][app] if r]

normalizedData = normalize(data)
if EXPERIMENT_TAG == 'harvard':
    cases = {int(id): getDataframe(case) for id, case in normalizedData.items() if int(id)>500}
else:
    cases = {int(id): getDataframe(case) for id, case in normalizedData.items() if int(id)<=500}

display(f'We have {len(cases)} cases in the experiment.')

'We have 44 cases in the experiment.'

In [4]:
caseClassificationOld = loadData("data/case-classification_old")
failures_temp = (
    pd.read_excel("data/AppTest-2024-02-06.xlsx")
    .dropna(subset=["invalid_code"])
    .groupby(["medical_app", "invalid_code"])["case_study"]
    .agg(set)
    .reset_index()
).set_index(["medical_app", "invalid_code"]).to_dict('index')

failures = defaultdict(dict)
for (app, err), failedCases in  failures_temp.items():
    failures[app][err] = failedCases['case_study']

for app in failures:
    failures[app]["session failed"] = set()
    for err, failedCases in failures[app].items():
        if err != 'NDX':
            failures[app]["session failed"] |= failedCases

    failures[app]["no disease found"] = failures[app]["NDX"]
    del failures[app]["NDX"]

failures = dict(failures)

if EXPERIMENT_TAG == 'avey':
    for app, data in caseClassificationOld['apps'].items():
        failures[app] = data 
# failures

In [5]:
df = pd.read_excel(
    "data/Results - Apps performance - team copy.xlsx",
    sheet_name=(
        "Harvard cases"
        if EXPERIMENT_TAG == "harvard"
        else "Detailed diseases coverage - 50"
    ),
    index_col=1,
).dropna(subset=["Common vs Less common"])["Common vs Less common"]

df.index = [int(str(i).replace('#','')) for i in df.index]
caseClassification = {
    'common':set(),
    'less common':set(),
    'apps':failures,
}
for case_num, typ in df.items():
    if typ == 'Common':
        caseClassification['common'].add(case_num)
    else:
        caseClassification['less common'].add(case_num)

assert set(cases.keys()).issubset(
    caseClassification["common"] | caseClassification["less common"]
), f'Vignette results have extra ids that are not in classification {set(cases.keys()) - (caseClassification["common"] | caseClassification["less common"])}'

print(f'We have {len(caseClassification["common"])} common diseases and {len(caseClassification["less common"])} less common diseases')
display('App v Error code: Case Count')
pd.DataFrame({app: {err: len(v) for err, v in errs.items()} for app, errs in caseClassification["apps"].items()}).transpose()

We have 38 common diseases and 7 less common diseases


'App v Error code: Case Count'

,MAC,OTH,session failed,no disease found,A01,CNA,A16,A18
Avey,3.0,1.0,4.0,29.0,NaN,NaN,NaN,NaN
Avey v2,NaN,1.0,1.0,13.0,NaN,NaN,NaN,NaN
Buoy,7.0,7.0,23.0,3.0,7.0,2.0,NaN,NaN
Healthily,NaN,NaN,64.0,18.0,NaN,5.0,59.0,NaN
K Health,10.0,3.0,95.0,1.0,NaN,19.0,NaN,63.0
Mediktor,1.0,NaN,1.0,41.0,NaN,NaN,NaN,NaN
Symptomate,1.0,NaN,9.0,17.0,NaN,NaN,NaN,8.0
WebMD,NaN,NaN,6.0,1.0,5.0,1.0,NaN,NaN


## TODO: NEED TO REMOVE DUPLICATE TESTS AND VERIFY THE NUMBER OF ERRORS

<p>

### Let us have a look at all the cases. <a class="anchor" id="cases"></a>
The cases have been labelled common and less common by our doctors. We have also labelled the cases with apps that failed on them.

In [6]:
from IPython.display import display

for caseNum, case in cases.items():
    isCommonString = 'common' if caseNum in caseClassification['common'] else 'less common'
    sessionFailed, noDiseaseFound = ([], [])
    for app, failedCases in caseClassification['apps'].items():
        if caseNum in failedCases['session failed']:
            sessionFailed.append(app)
        elif caseNum in failedCases['no disease found']:
            noDiseaseFound.append(app)

    print(f'Case number {caseNum} ({isCommonString})')
    if sessionFailed:
        print(
            f'Session failed to start for: {", ".join(sessionFailed)}.')
    if noDiseaseFound:
        print(
            f'No diseases were found for: {", ".join(noDiseaseFound)}.')
    display(case)
    case.to_excel(f'individual_case_outputs/{EXPERIMENT_TAG}/{caseNum}.xlsx')
    print('\n'*2)


Case number 517 (common)
Session failed to start for: Healthily, K Health.


,gold,Ada,Avey,Avey v2,Buoy,ChatGPT - 4,Healthily,K Health,Mediktor,Symptomate,WebMD,doctor_MA,doctor_NJ,doctor_TH
1,acute pharyngitis,tosillitis,acute pharyngitis,acute pharyngitis,influenza,acute pharyngitis,None,None,esophageal foreign body,acute pharyngitis,influenza,tonsils,gas tonsillitis,acute pharyngitis
2,None,scarlet fever,diphtheria,diphtheria,coronavirus (covid-19),influenza,None,None,acute pharyngitis,viral gastroenteritis,swine influenza,acute pharyngitis,None,None
3,None,glandular fever,infectious mononucleosis,infectious mononucleosis,acute pharyngitis,None,None,None,mononucleosis,pneumonia,acute appendicitis,None,None,None
4,None,cytomegalovirus infection,covid 19,epiglottitis,None,None,None,None,None,infectious mononucleosis,bacterial pneumonia,None,None,None
5,None,None,None,covid 19,None,None,None,None,None,scarlet fever,acute pharyngitis,None,None,None
6,None,None,None,None,None,None,None,None,None,None,septicemia,None,None,None
7,None,None,None,None,None,None,None,None,None,None,peritonitis,None,None,None
8,None,None,None,None,None,None,None,None,None,None,pyelonephritis,None,None,None





Case number 518 (common)
Session failed to start for: Avey.


,gold,Ada,Avey,Avey v2,Buoy,ChatGPT - 4,Healthily,K Health,Mediktor,Symptomate,WebMD,doctor_MA,doctor_NJ,doctor_TH
1,acute pharyngitis,bacterial tonsilitis,None,acute pharyngitis,covid 19,acute pharyngitis,quinzy,infectious mononucleosis,acute pharyngitis,acute pharyngitis,acute pharyngitis,tonsillitis,tonsillitis/ start treatment,acute pharyngitis
2,None,acute pharyngitis,None,diphtheria,infectious mononucleosis,gastroesophageal reflux disease,acute pharyngitis,acute pharyngitis,infectious mononucleosis,None,bacterial tonsilitis,acute pharyngitis,None,None
3,None,glandular fever,None,infectious mononucleosis,None,bacterial tonsilitis,None,None,peritonsillar abscesses,None,adenovirus infection,None,None,None
4,None,quinzy,None,acute laryngitis,None,None,None,None,None,None,common cold,None,None,None
5,None,None,None,epiglottitis,None,None,None,None,None,None,flu,None,None,None
6,None,None,None,None,None,None,None,None,None,None,infectious mononucleosis,None,None,None
7,None,None,None,None,None,None,None,None,None,None,chronic sinusitis,None,None,None





Case number 521 (common)
Session failed to start for: Avey.


...

,gold,Ada,Avey,Avey v2,Buoy,ChatGPT - 4,Healthily,K Health,Mediktor,Symptomate,WebMD,doctor_MA,doctor_NJ,doctor_TH
1,stye,stye,stye,stye,stye,orbital cellulitis,stye,stye,None,stye,acute sinusitis,stye,stye,stye
2,None,bacterial conjunctivitis,lipoma,lipoma,None,stye,periorbital cellulitis,bacterial conjunctivitis,None,periorbital cellulitis,bacterial conjunctivitis,calazion,blepharitis,None
3,None,shingles,chalazion,chalazion,None,chalazion,None,blepharitis,None,None,chronic sinusitis,None,None,None
4,None,None,blepharitis,blepharitis,None,None,None,episcleritis,None,None,cluster headache,None,None,None
5,None,None,None,None,None,None,None,None,None,None,allergic conjunctivitis,None,None,None





Case number 544 (common)


,gold,Ada,Avey,Avey v2,Buoy,ChatGPT - 4,Healthily,K Health,Mediktor,Symptomate,WebMD,doctor_MA,doctor_NJ,doctor_TH
1,viral upper respiratory infection,viral upper respiratory infection,viral upper respiratory infection,viral upper respiratory infection,viral upper respiratory infection,viral upper respiratory infection,viral upper respiratory infection,viral upper respiratory infection,viral upper respiratory infection,viral upper respiratory infection,viral upper respiratory infection,viral upper respiratory infection,viral upper respiratory infection,viral upper respiratory infection
2,None,None,diphtheria,acute bronchitis,None,allergic rhinitis,None,None,None,acute streptococcal pharyngitis,hay fever,None,None,None
3,None,None,acute bronchitis,diphtheria,None,None,None,None,None,mononucleosis,bronchitis,None,None,None
4,None,None,None,None,None,None,None,None,None,None,chronic sinusitis,None,None,None





Case number 522 (common)


,gold,Ada,Avey,Avey v2,Buoy,ChatGPT - 4,Healthily,K Health,Mediktor,Symptomate,WebMD,doctor_MA,doctor_NJ,doctor_TH
1,copd flare (milder),acute bronchitis,copd flare (milder),copd flare (milder),copd flare (milder),copd flare (milder),acute bronchitis,asthma,copd flare (milder),asthma,asthma,acute bronchitis on top of chronic bronchitis,copd flare (milder),copd flare (milder)
2,None,asthma,asthma,asthma,asthma,asthma,None,upper respiratory infection,typical pneumonia,None,copd flare (milder),copd flare (milder),acute rhinosinusitis with post nasal drip,flu
3,None,occupational asthma,bronchiectasis,bronchiectasis,covid 19,lung cancer,None,pneumonia,asthma,None,congestive heart failure,asthma,None,None
4,None,pneumonia,cystic fibrosis,cystic fibrosis,None,chronic bronchitis,None,None,foreign body aspiration,None,diastolic heart failure,None,None,None
5,None,covid 19,upper airway obstruction,heart failure,None,emphysema,None,None,heart failure,None,bacterial pneumonia,None,None,None
6,None,None,None,None,None,None,None,None,lung cancer,None,pulmonary embolism,None,None,None
7,None,None,None,None,None,None,None,None,bronchial hyperresponsiveness,None,influenza,None,None,None
8,None,None,None,None,None,None,None,None,aspiration pneumonia,None,None,None,None,None
9,None,None,None,None,None,None,None,None,tuberculosis,None,None,None,None,None
10,None,None,None,None,None,None,None,None,biventricular heart failure,None,None,None,None,None


<p>

## Let us define the metrics now. <a class="anchor" id="metrics"></a>

### Terms used
- TP: True positive (correct disease retrieved)
- TN: True negative (wrong disease **not** retrieved)
- FP: False positive (wrong disease retrieved)
- FN: False negative (correct disease **not** retrieved)
- gold standard - the correct list of diseases as determined by collective intelligence of doctors

### Precision
Precision helps us understand how exact our results are. It gives us an intuition about how many wrong diseases (false positives) are being retrieved. It is the ratio *number of correct diseases retrieved* to the *length of the complete list retrieved*.
$$precision = \frac{TP}{TP + FP} = \frac{TP}{\text{length of differential list}}$$

### Recall
Recall is a measure of how many of the correct diseases are being retrieved. It is the ratio *number of correct diseases retrieved* to the *length of the gold standard list*.
$$recall = \frac{TP}{TP + FN} = \frac{TP}{\text{length of the gold standard}}$$

### F1 Score
F1 score is the weighted harmonic mean of *precision* and *recall*. It is a metric that combines *precision* and *recall* and gives us 1 score for easier comparison.

Suppose $\beta$ defines how important is $recall$ to $precision$ then,
$$fscore_{\beta} = (1 + \beta^2)\frac{precision \cdot recall}{(\beta^2 \cdot precision) + recall}$$
Substituting $\beta = 1$,
$$fscore_{1} = \frac{2 \cdot precision \cdot recall}{ precision + recall}$$

### NDCG
NDCG or Normalized Discounted Cumulative Gain is measure of how accurate the ranking is. In our calculations, we use
$$DCG = \sum_{i=1}^n\frac{2^{relevance_i}-1}{log_2(i+1)}$$
where $n$ is the number of differentials in the returned list and  
$relevance_i = |gold standard| - rank_{gold\ standard}(ddx[i])$ if $ddx[i]$ is present, 0 otherwise.

$$NDCG = \frac{DCG_{ddx}}{DCG_{gold\ standard}}$$

### M Score
M Score determines where the top disease (gold standard) appears in the returned differential.
$$M_i = \text{gold standard[0]} \in \text{ddx[:i]}$$

### Position
Shows the position of the gold standard[0] in the returned differential. 

### Length
$$length = \frac{|ddx|}{|gold\ standard|}$$
<br>
<br>
<br>
<br>


In [7]:
import math

def getPrecision(goldStandard: pd.Series, candidate: pd.Series) -> float:
    tp = sum(int(disease in goldStandard.values and disease is not None)
             for disease in candidate)
    return tp if tp == 0 else tp/candidate.count()


def getRecall(goldStandard: pd.Series, candidate: pd.Series) -> float:
    tp = sum(int(disease in goldStandard.values and disease is not None)
             for disease in candidate)
    return tp/goldStandard.count()


def getFScore(precision: float, recall: float, beta: float = 1) -> float:
    return math.nan if precision+recall == 0 else \
        (1+beta**2)*precision*recall/(precision*(beta**2)+recall)


def getNDCG(goldStandard: pd.Series, candidate: pd.Series, scores) -> float:
    def discount(score: float, index: int) -> float:
        '''The index is 1 based'''
        return (math.pow(2, score)-1)/math.log2(index+1)

    maxDCG = sum(discount(scores[i], i+1) for i in range(len(scores)))

    candidateRelevance = []
    goldStandard = list(goldStandard)
    for index, disease in enumerate(candidate):
        if disease is not None and disease in goldStandard:
            candidateRelevance.append(
                discount(scores[goldStandard.index(disease)], index+1))
        else:
            candidateRelevance.append(0)

    return sum(candidateRelevance)/maxDCG


def getMScore(goldStandard: pd.Series, candidate: pd.Series, m=1) -> bool:
    return goldStandard.values[0] in candidate.values[:m]


def getPosition(goldStandard: pd.Series, candidate: pd.Series) -> bool:
    return math.nan if goldStandard.values[0] not in candidate.values else\
        1 + list(candidate.values).index(goldStandard.values[0])


def getLength(goldStandard: pd.Series, candidate: pd.Series) -> int:
    return math.nan if candidate.count() == 0 else \
        candidate.count()/goldStandard.count()


def getScoresCase(id,case: pd.DataFrame) -> pd.DataFrame:
    gold = case['gold']
    if EXPERIMENT_TAG == 'avey':
        gold_old = case['gold_old']
    else:
        gold_old = gold

    scores = [
        [getPrecision(gold_old if '_old' in col else gold, case[col]) for col in case.columns if "gold" not in col],
        [getRecall(gold_old if '_old' in col else gold, case[col]) for col in case.columns if "gold" not in col],
    ]

    scores.append(
        [getFScore(prec, rec, beta=1) for prec, rec in zip(scores[0], scores[1])]
    )
    scores.append(
        [getFScore(prec, rec, beta=2) for prec, rec in zip(scores[0], scores[1])]
    )

    # relevance for a list of 4 is 4, 3, 2, 1
    # relevance for a list of 2 is 2, 1
    scores.append(
        [
            getNDCG(
                gold_old if "_old" in col else gold,
                case[col],
                list(range((gold_old if "_old" in col else gold).count(), 0, -1)),
            )
            for col in case.columns
            if "gold" not in col
        ]
    )

    for m in range(1, 6, 2):
        scores.append(
            [
                getMScore(gold_old if "_old" in col else gold, case[col], m)
                for col in case.columns
                if "gold" not in col
            ]
        )

    try:
        scores.append(
            [
                getPosition(gold_old if "_old" in col else gold, case[col])
                for col in case.columns
                if "gold" not in col
            ]
        )
    except Exception as e:
        print(id,flush=True)
        raise e

    scores.append(
        [
            getLength(gold_old if "_old" in col else gold, case[col])
            for col in case.columns
            if "gold" not in col
        ]
    )

    return pd.DataFrame(
        scores,
        columns=[col for col in case.columns if "gold" not in col],
        index=[
            "precision",
            "recall",
            "f1-score",
            "f2-score",
            "NDCG",
            "M1",
            "M3",
            "M5",
            "position",
            "length (x of gs)",
        ],
    )

scores = {}
for id, case in cases.items():
    try: scores[id] = getScoresCase(id, case)
    except Exception as e:
        print(id,flush=True)
        raise e

# scores = {id: getScoresCase(id, case) for id, case in cases.items()}

In [8]:
for id, score in scores.items():
    score.to_excel(f'individual_results/{EXPERIMENT_TAG}/{id}.xlsx')

In [9]:
print(len(scores))

44


Let us define the experiment now. We will pick which cases to compute startistics for.

In [10]:
from enum import Enum 

class DiseaseType(Enum):
    ALL = 'all'
    COMMON = 'common-diseases'
    UNCOMMON = 'less-common-diseases'

class FailureType(Enum):
    ANY = 'any-error'
    NO_DDX = 'no-disease-found'
    SES_FAIL = 'session-failed'
    NONE = 'None'

In [11]:
from collections import defaultdict

experiments = {}

def getExpName(disease_type:DiseaseType,failure_type:FailureType=FailureType.NONE,app='NA'):
    return f"disease_type={disease_type.value} failure-type-ignored={failure_type.value} for app={app}"

def addExperiment(disease_type, casesToConsider):
    # add experiment to ignore no case
    experiments[getExpName(disease_type)] = set(casesToConsider)

    caseClassificationApps = {}
    for app, failure_type in caseClassification["apps"].items():
        caseClassificationApps[app] = {
            FailureType.NO_DDX: failure_type["no disease found"],
            FailureType.SES_FAIL: failure_type["session failed"],
            FailureType.ANY: set(failure_type["no disease found"])
            | set(failure_type["session failed"]),
        }

    for app, classifications in caseClassificationApps.items():
        for failure_type, caseNums in classifications.items():

            # experiment to ignore the cases for the particular app
            experiments[getExpName(disease_type,failure_type,app)] = set(casesToConsider) - set(caseNums)

            # experiment to ignore the cases for all the apps
            if getExpName(disease_type,failure_type,'any') in experiments:
                experiments[getExpName(disease_type,failure_type,'any')] = (
                    experiments[getExpName(disease_type,failure_type,'any')]
                    - set(caseNums)
                )
            else:
                experiments[getExpName(disease_type,failure_type,'any')] = experiments[getExpName(disease_type,failure_type,app)]


addExperiment(DiseaseType.COMMON, caseClassification["common"])
addExperiment(DiseaseType.UNCOMMON, caseClassification["less common"])
addExperiment(
    DiseaseType.ALL,
    set(caseClassification["less common"]) | set(caseClassification["common"]),
)

display("The experiments we are going to conduct are:")
dict(enumerate(sorted(list(experiments.keys()))))

'The experiments we are going to conduct are:'

{0: 'disease_type=all failure-type-ignored=None for app=NA',
 1: 'disease_type=all failure-type-ignored=any-error for app=Avey',
 2: 'disease_type=all failure-type-ignored=any-error for app=Avey v2',
 3: 'disease_type=all failure-type-ignored=any-error for app=Buoy',
 4: 'disease_type=all failure-type-ignored=any-error for app=Healthily',
 5: 'disease_type=all failure-type-ignored=any-error for app=K Health',
 6: 'disease_type=all failure-type-ignored=any-error for app=Mediktor',
 7: 'disease_type=all failure-type-ignored=any-error for app=Symptomate',
 8: 'disease_type=all failure-type-ignored=any-error for app=WebMD',
 9: 'disease_type=all failure-type-ignored=any-error for app=any',
 10: 'disease_type=all failure-type-ignored=no-disease-found for app=Avey',
 11: 'disease_type=all failure-type-ignored=no-disease-found for app=Avey v2',
 12: 'disease_type=all failure-type-ignored=no-disease-found for app=Buoy',
 13: 'disease_type=all failure-type-ignored=no-disease-found for app=Healt

<br>
<br>

### Results <a class="anchor" id="results"></a>

In [12]:
#shows the number of cases where Avey failed M1
#Do not run this after Avey v2 is replaces Avey

count = 0
for id, score in scores.items():
    if not score.loc["M1","Avey v2"]:
        print(id)
        count += 1 

count

521
527
534
503
524
526
545
501
506
502
505
507
509
516
519
520
537
539
540


19

In [13]:
def getStats(scores, row: str, col: str):
    values = []
    for score in scores.values():
        try:
            if not math.isnan(score.loc[row, col]):
                values.append(score.loc[row, col])
        except KeyError as e:
            if '_old' not in col:
                display(score,row,col)
                raise e
    if not values:
        return {
            (col, "average"): 0,
            (col, "variance"): 0,
            (col, "std Dev"): 0,
        }
    
    average = sum(values)/len(values)
    variance = sum((value-average)**2 for value in values)/len(values)
    stdDev = variance**0.5
    return {
        (col,"average"): round(average, 3), 
        (col,"variance"):round(variance, 3), 
        (col,"std Dev"):round(stdDev, 3),
        }


results = {}
for label, casesToConsider in experiments.items():
    # selectedScores = {id: score for id,
    #                   score in scores.items()}
    selectedScores = {id: score for id,
                      score in scores.items() if int(id) in casesToConsider}
    
    columns = next(iter(scores.values())).columns
    index = [
            "precision",
            "recall",
            "f1-score",
            "f2-score",
            "NDCG",
            "M1",
            "M3",
            "M5",
            "position",
            "length (x of gs)",
    ]

    metrics = defaultdict(list)
    for row in index:
        for stat in [getStats(selectedScores, row, col) for col in columns]:
            for key,val in stat.items():
                metrics[key].append(val)

    averageScores = pd.DataFrame(metrics, index=[f"stats_for_{x}" for x in index])

    # ignore nan for recall
    for col in columns:
        p = averageScores.loc["stats_for_precision", (col,'average')]
        r = averageScores.loc["stats_for_recall", (col, "average")]
        averageScores.loc["stats_for_f1-score", (col, "average")] = round(getFScore(p, r, 1), 3)
        averageScores.loc["stats_for_f2-score", (col, "average")] = round(getFScore(p, r, 2), 3)

        for key in ['variance','std Dev']:
            averageScores.loc["stats_for_f1-score", (col, key)] = np.nan
            averageScores.loc["stats_for_f2-score", (col, key)] = np.nan

    def calcStats(doctorResults):
        def extractItem(data, col='average'):
            return data[col]

        sum = pd.Series(extractItem(doctorResults[0]))
        for data in doctorResults[1:]:
            sum += extractItem(data)
        average = (sum/len(doctorResults)).round(3)
        return average

    doctorResultsAverage = calcStats(
        [
            averageScores.loc[:, "doctor_MA"] ,
            averageScores.loc[:, "doctor_NJ"] ,
            averageScores.loc[:, "doctor_TH"]
        ]
    )

    if EXPERIMENT_TAG != 'harvard':
        doctorResultsAverageOld = calcStats(
            [
                averageScores.loc[:, "doctor_MA_old"] ,
                averageScores.loc[:, "doctor_NJ_old"] ,
                averageScores.loc[:, "doctor_TH_old"]
            ]
        )
        averageScores["average_doctor_old"] = doctorResultsAverageOld

    # averageScores.insert(
    #     loc=8, column="average_doctor",
    #     value=doctorResultsAverage,
    # )

    averageScores["average_doctor"] = doctorResultsAverage

    results[label] = averageScores

In [14]:
def displayResults(results,printNumCases=True):
    for label, result in results.items():
        if printNumCases:
            print(f'Results for experiment `{label}`, which has {len(set(experiments[label]) & set(scores.keys()))} cases, is')
        else:
            print(f'Results for experiment `{label}` is')
        display(result)
        result.to_excel(f'stats/{EXPERIMENT_TAG}/{label}.xlsx')

displayResults({key:val for key, val in results.items()})

Results for experiment `disease_type=common-diseases failure-type-ignored=None for app=NA`, which has 37 cases, is


Ada                     Avey                   \
                           average variance std Dev average variance std Dev   
stats_for_precision          0.291    0.052   0.227   0.149    0.013   0.115   
stats_for_recall             0.811    0.153   0.392   0.649    0.228   0.477   
stats_for_f1-score           0.428      NaN     NaN   0.242      NaN     NaN   
stats_for_f2-score           0.597      NaN     NaN   0.388      NaN     NaN   
stats_for_NDCG               0.744    0.152   0.390   0.536    0.190   0.436   
stats_for_M1                 0.649    0.228   0.477   0.405    0.241   0.491   
stats_for_M3                 0.811    0.153   0.392   0.568    0.245   0.495   
stats_for_M5                 0.811    0.153   0.392   0.649    0.228   0.477   
stats_for_position           1.267    0.329   0.573   1.708    1.123   1.060   
stats_for_length (x of gs)   3.514    1.439   1.200   4.613    0.431   0.656   

                           Avey v2                     Buoy  ... doctor_MA  \
                           average variance std Dev average  ...   average   
stats_for_precision          0.187    0.012   0.109   0.338  ...     2.074   
stats_for_recall             0.811    0.153   0.392   0.649  ...     2.649   
stats_for_f1-score           0.304      NaN     NaN   0.445  ...     2.324   
stats_for_f2-score           0.486      NaN     NaN   0.548  ...     2.508   
stats_for_NDCG               0.695    0.154   0.392   0.575  ...     2.526   
stats_for_M1                 0.568    0.245   0.495   0.486  ...     2.325   
stats_for_M3                 0.730    0.197   0.444   0.649  ...     2.649   
stats_for_M5                 0.811    0.153   0.392   0.649  ...     2.649   
stats_for_position           1.633    1.366   1.169   1.417  ...     3.374   
stats_for_length (x of gs)   4.629    0.633   0.796   2.273  ...     4.514   

                                            doctor_NJ                   \
                           variance std Dev   average variance std Dev   
stats_for_precision           0.095   0.308     0.649    0.174   0.417   
stats_for_recall              0.026   0.162     0.757    0.184   0.429   
stats_for_f1-score              NaN     NaN     0.699      NaN     NaN   
stats_for_f2-score              NaN     NaN     0.733      NaN     NaN   
stats_for_NDCG                0.047   0.218     0.747    0.183   0.428   
stats_for_M1                  0.184   0.429     0.730    0.197   0.444   
stats_for_M3                  0.026   0.162     0.757    0.184   0.429   
stats_for_M5                  0.026   0.162     0.757    0.184   0.429   
stats_for_position            0.243   0.493     1.036    0.034   0.186   
stats_for_length (x of gs)    0.722   0.850     1.297    0.371   0.609   

                           doctor_TH                  average_doctor  
                             average variance std Dev                 
stats_for_precision            0.695    0.115   0.340          0.691  
stats_for_recall               0.919    0.075   0.273          0.883  
stats_for_f1-score             0.791      NaN     NaN          0.775  
stats_for_f2-score             0.863      NaN     NaN          0.836  
stats_for_NDCG                 0.889    0.080   0.282          0.842  
stats_for_M1                   0.838    0.136   0.369          0.775  
stats_for_M3                   0.919    0.075   0.273          0.883  
stats_for_M5                   0.919    0.075   0.273          0.883  
stats_for_position             1.088    0.080   0.284          1.125  
stats_for_length (x of gs)     1.595    0.728   0.853          1.505  

[10 rows x 40 columns]

Results for experiment `disease_type=common-diseases failure-type-ignored=no-disease-found for app=Avey`, which has 33 cases, is


Ada                     Avey                   \
                           average variance std Dev average variance std Dev   
stats_for_precision          0.301    0.057   0.239   0.167    0.012   0.109   
stats_for_recall             0.788    0.167   0.409   0.727    0.198   0.445   
stats_for_f1-score           0.436      NaN     NaN   0.272      NaN     NaN   
stats_for_f2-score           0.595      NaN     NaN   0.435      NaN     NaN   
stats_for_NDCG               0.750    0.166   0.407   0.600    0.174   0.417   
stats_for_M1                 0.697    0.211   0.460   0.455    0.248   0.498   
stats_for_M3                 0.788    0.167   0.409   0.636    0.231   0.481   
stats_for_M5                 0.788    0.167   0.409   0.727    0.198   0.445   
stats_for_position           1.154    0.207   0.455   1.708    1.123   1.060   
stats_for_length (x of gs)   3.364    1.383   1.176   4.613    0.431   0.656   

                           Avey v2                     Buoy  ... doctor_MA  \
                           average variance std Dev average  ...   average   
stats_for_precision          0.198    0.011   0.105   0.369  ...     2.006   
stats_for_recall             0.848    0.129   0.359   0.697  ...     2.606   
stats_for_f1-score           0.321      NaN     NaN   0.483  ...     2.264   
stats_for_f2-score           0.512      NaN     NaN   0.592  ...     2.456   
stats_for_NDCG               0.730    0.139   0.373   0.614  ...     2.490   
stats_for_M1                 0.606    0.239   0.489   0.515  ...     2.303   
stats_for_M3                 0.758    0.184   0.429   0.697  ...     2.606   
stats_for_M5                 0.848    0.129   0.359   0.697  ...     2.606   
stats_for_position           1.643    1.444   1.202   1.435  ...     3.350   
stats_for_length (x of gs)   4.606    0.663   0.814   2.310  ...     4.606   

                                            doctor_NJ                   \
                           variance std Dev   average variance std Dev   
stats_for_precision           0.098   0.313     0.621    0.182   0.427   
stats_for_recall              0.029   0.171     0.727    0.198   0.445   
stats_for_f1-score              NaN     NaN     0.670      NaN     NaN   
stats_for_f2-score              NaN     NaN     0.703      NaN     NaN   
stats_for_NDCG                0.050   0.224     0.727    0.198   0.445   
stats_for_M1                  0.184   0.429     0.727    0.198   0.445   
stats_for_M3                  0.029   0.171     0.727    0.198   0.445   
stats_for_M5                  0.029   0.171     0.727    0.198   0.445   
stats_for_position            0.250   0.500     1.000    0.000   0.000   
stats_for_length (x of gs)    0.768   0.876     1.303    0.393   0.627   

                           doctor_TH                  average_doctor  
                             average variance std Dev                 
stats_for_precision            0.673    0.119   0.345          0.669  
stats_for_recall               0.909    0.083   0.287          0.869  
stats_for_f1-score             0.773      NaN     NaN          0.755  
stats_for_f2-score             0.849      NaN     NaN          0.819  
stats_for_NDCG                 0.876    0.088   0.296          0.830  
stats_for_M1                   0.818    0.149   0.386          0.768  
stats_for_M3                   0.909    0.083   0.287          0.869  
stats_for_M5                   0.909    0.083   0.287          0.869  
stats_for_position             1.100    0.090   0.300          1.117  
stats_for_length (x of gs)     1.636    0.777   0.881          1.535  

[10 rows x 40 columns]

Results for experiment `disease_type=common-diseases failure-type-ignored=no-disease-found for app=any`, which has 30 cases, is


...

Ada                     Avey                   \
                           average variance std Dev average variance std Dev   
stats_for_precision          0.284    0.046   0.214   0.166    0.029   0.169   
stats_for_recall             0.818    0.149   0.386   0.659    0.225   0.474   
stats_for_f1-score           0.422      NaN     NaN   0.265      NaN     NaN   
stats_for_f2-score           0.594      NaN     NaN   0.413      NaN     NaN   
stats_for_NDCG               0.736    0.151   0.388   0.553    0.192   0.439   
stats_for_M1                 0.636    0.231   0.481   0.432    0.245   0.495   
stats_for_M3                 0.773    0.176   0.419   0.591    0.242   0.492   
stats_for_M5                 0.818    0.149   0.386   0.659    0.225   0.474   
stats_for_position           1.389    0.682   0.826   1.655    1.054   1.026   
stats_for_length (x of gs)   3.568    1.382   1.175   4.579    0.717   0.847   

                           Avey v2                     Buoy  ... doctor_MA  \
                           average variance std Dev average  ...   average   
stats_for_precision          0.198    0.027   0.163   0.303  ...     2.018   
stats_for_recall             0.795    0.163   0.403   0.591  ...     2.637   
stats_for_f1-score           0.317      NaN     NaN   0.401  ...     2.284   
stats_for_f2-score           0.496      NaN     NaN   0.497  ...     2.483   
stats_for_NDCG               0.690    0.160   0.400   0.520  ...     2.516   
stats_for_M1                 0.568    0.245   0.495   0.432  ...     2.318   
stats_for_M3                 0.727    0.198   0.445   0.591  ...     2.637   
stats_for_M5                 0.795    0.163   0.403   0.591  ...     2.637   
stats_for_position           1.571    1.216   1.103   1.423  ...     3.367   
stats_for_length (x of gs)   4.537    0.932   0.965   2.308  ...     4.705   

                                            doctor_NJ                   \
                           variance std Dev   average variance std Dev   
stats_for_precision           0.106   0.326     0.648    0.166   0.407   
stats_for_recall              0.043   0.208     0.773    0.176   0.419   
stats_for_f1-score              NaN     NaN     0.705      NaN     NaN   
stats_for_f2-score              NaN     NaN     0.744      NaN     NaN   
stats_for_NDCG                0.061   0.246     0.764    0.175   0.418   
stats_for_M1                  0.188   0.433     0.750    0.188   0.433   
stats_for_M3                  0.043   0.208     0.773    0.176   0.419   
stats_for_M5                  0.043   0.208     0.773    0.176   0.419   
stats_for_position            0.229   0.479     1.029    0.029   0.169   
stats_for_length (x of gs)    0.808   0.899     1.318    0.353   0.594   

                           doctor_TH                  average_doctor  
                             average variance std Dev                 
stats_for_precision            0.664    0.121   0.348          0.673  
stats_for_recall               0.909    0.083   0.287          0.879  
stats_for_f1-score             0.767      NaN     NaN          0.761  
stats_for_f2-score             0.847      NaN     NaN          0.828  
stats_for_NDCG                 0.876    0.088   0.296          0.839  
stats_for_M1                   0.818    0.149   0.386          0.773  
stats_for_M3                   0.909    0.083   0.287          0.879  
stats_for_M5                   0.909    0.083   0.287          0.879  
stats_for_position             1.100    0.090   0.300          1.122  
stats_for_length (x of gs)     1.705    0.799   0.894          1.568  

[10 rows x 40 columns]

Results for experiment `disease_type=all failure-type-ignored=session-failed for app=WebMD`, which has 43 cases, is


Ada                     Avey                   \
                           average variance std Dev average variance std Dev   
stats_for_precision          0.279    0.046   0.214   0.164    0.029   0.170   
stats_for_recall             0.814    0.151   0.389   0.651    0.227   0.477   
stats_for_f1-score           0.416      NaN     NaN   0.262      NaN     NaN   
stats_for_f2-score           0.588      NaN     NaN   0.408      NaN     NaN   
stats_for_NDCG               0.738    0.154   0.392   0.551    0.197   0.443   
stats_for_M1                 0.651    0.227   0.477   0.442    0.247   0.497   
stats_for_M3                 0.767    0.178   0.422   0.581    0.243   0.493   
stats_for_M5                 0.814    0.151   0.389   0.651    0.227   0.477   
stats_for_position           1.371    0.691   0.831   1.643    1.087   1.042   
stats_for_length (x of gs)   3.605    1.355   1.164   4.595    0.728   0.853   

                           Avey v2                     Buoy  ... doctor_MA  \
                           average variance std Dev average  ...   average   
stats_for_precision          0.197    0.027   0.164   0.310  ...     2.030   
stats_for_recall             0.791    0.165   0.407   0.605  ...     2.651   
stats_for_f1-score           0.315      NaN     NaN   0.410  ...     2.297   
stats_for_f2-score           0.493      NaN     NaN   0.508  ...     2.497   
stats_for_NDCG               0.691    0.163   0.404   0.532  ...     2.528   
stats_for_M1                 0.581    0.243   0.493   0.442  ...     2.325   
stats_for_M3                 0.721    0.201   0.449   0.605  ...     2.651   
stats_for_M5                 0.791    0.165   0.407   0.605  ...     2.651   
stats_for_position           1.559    1.247   1.116   1.423  ...     3.376   
stats_for_length (x of gs)   4.550    0.948   0.973   2.308  ...     4.722   

                                            doctor_NJ                   \
                           variance std Dev   average variance std Dev   
stats_for_precision           0.107   0.327     0.663    0.160   0.399   
stats_for_recall              0.044   0.211     0.791    0.165   0.407   
stats_for_f1-score              NaN     NaN     0.721      NaN     NaN   
stats_for_f2-score              NaN     NaN     0.762      NaN     NaN   
stats_for_NDCG                0.062   0.248     0.782    0.165   0.406   
stats_for_M1                  0.190   0.436     0.767    0.178   0.422   
stats_for_M3                  0.044   0.211     0.791    0.165   0.407   
stats_for_M5                  0.044   0.211     0.791    0.165   0.407   
stats_for_position            0.233   0.483     1.029    0.029   0.169   
stats_for_length (x of gs)    0.816   0.903     1.326    0.359   0.599   

                           doctor_TH                  average_doctor  
                             average variance std Dev                 
stats_for_precision            0.667    0.123   0.351          0.677  
stats_for_recall               0.907    0.084   0.290          0.884  
stats_for_f1-score             0.769      NaN     NaN          0.766  
stats_for_f2-score             0.846      NaN     NaN          0.832  
stats_for_NDCG                 0.873    0.089   0.299          0.843  
stats_for_M1                   0.814    0.151   0.389          0.775  
stats_for_M3                   0.907    0.084   0.290          0.884  
stats_for_M5                   0.907    0.084   0.290          0.884  
stats_for_position             1.103    0.092   0.303          1.125  
stats_for_length (x of gs)     1.698    0.816   0.903          1.574  

[10 rows x 40 columns]

Results for experiment `disease_type=all failure-type-ignored=any-error for app=WebMD`, which has 43 cases, is


Ada                     Avey                   \
                           average variance std Dev average variance std Dev   
stats_for_precision          0.279    0.046   0.214   0.164    0.029   0.170   
stats_for_recall             0.814    0.151   0.389   0.651    0.227   0.477   
stats_for_f1-score           0.416      NaN     NaN   0.262      NaN     NaN   
stats_for_f2-score           0.588      NaN     NaN   0.408      NaN     NaN   
stats_for_NDCG               0.738    0.154   0.392   0.551    0.197   0.443   
stats_for_M1                 0.651    0.227   0.477   0.442    0.247   0.497   
stats_for_M3                 0.767    0.178   0.422   0.581    0.243   0.493   
stats_for_M5                 0.814    0.151   0.389   0.651    0.227   0.477   
stats_for_position           1.371    0.691   0.831   1.643    1.087   1.042   
stats_for_length (x of gs)   3.605    1.355   1.164   4.595    0.728   0.853   

                           Avey v2                     Buoy  ... doctor_MA  \
                           average variance std Dev average  ...   average   
stats_for_precision          0.197    0.027   0.164   0.310  ...     2.030   
stats_for_recall             0.791    0.165   0.407   0.605  ...     2.651   
stats_for_f1-score           0.315      NaN     NaN   0.410  ...     2.297   
stats_for_f2-score           0.493      NaN     NaN   0.508  ...     2.497   
stats_for_NDCG               0.691    0.163   0.404   0.532  ...     2.528   
stats_for_M1                 0.581    0.243   0.493   0.442  ...     2.325   
stats_for_M3                 0.721    0.201   0.449   0.605  ...     2.651   
stats_for_M5                 0.791    0.165   0.407   0.605  ...     2.651   
stats_for_position           1.559    1.247   1.116   1.423  ...     3.376   
stats_for_length (x of gs)   4.550    0.948   0.973   2.308  ...     4.722   

                                            doctor_NJ                   \
                           variance std Dev   average variance std Dev   
stats_for_precision           0.107   0.327     0.663    0.160   0.399   
stats_for_recall              0.044   0.211     0.791    0.165   0.407   
stats_for_f1-score              NaN     NaN     0.721      NaN     NaN   
stats_for_f2-score              NaN     NaN     0.762      NaN     NaN   
stats_for_NDCG                0.062   0.248     0.782    0.165   0.406   
stats_for_M1                  0.190   0.436     0.767    0.178   0.422   
stats_for_M3                  0.044   0.211     0.791    0.165   0.407   
stats_for_M5                  0.044   0.211     0.791    0.165   0.407   
stats_for_position            0.233   0.483     1.029    0.029   0.169   
stats_for_length (x of gs)    0.816   0.903     1.326    0.359   0.599   

                           doctor_TH                  average_doctor  
                             average variance std Dev                 
stats_for_precision            0.667    0.123   0.351          0.677  
stats_for_recall               0.907    0.084   0.290          0.884  
stats_for_f1-score             0.769      NaN     NaN          0.766  
stats_for_f2-score             0.846      NaN     NaN          0.832  
stats_for_NDCG                 0.873    0.089   0.299          0.843  
stats_for_M1                   0.814    0.151   0.389          0.775  
stats_for_M3                   0.907    0.084   0.290          0.884  
stats_for_M5                   0.907    0.084   0.290          0.884  
stats_for_position             1.103    0.092   0.303          1.125  
stats_for_length (x of gs)     1.698    0.816   0.903          1.574  

[10 rows x 40 columns]

Now let us combine the individual results of the apps. We take the app results from different experiments and combine them. Our goal is to get the best results for each app and compare them.

In [15]:
bestResults = defaultdict(lambda : list(results.values())[0].copy())

for app in caseClassification['apps']:
    try:
        bestResults[getExpName(DiseaseType.ALL, FailureType.ANY, "best")][app] = results[
            getExpName(DiseaseType.ALL, FailureType.ANY, app)][app]
        bestResults[getExpName(DiseaseType.ALL, FailureType.SES_FAIL, "best")][app] = results[
            getExpName(DiseaseType.ALL, FailureType.ANY, app)][app]
        bestResults[getExpName(DiseaseType.ALL, FailureType.NO_DDX, "best")][app] = results[
            getExpName(DiseaseType.ALL, FailureType.ANY, app)][app]
    except Exception as e:
        display(getExpName(DiseaseType.ALL, FailureType.ANY, app))
        display(results[getExpName(DiseaseType.ALL, FailureType.ANY, app)])
        raise e

In [16]:
displayResults(bestResults,printNumCases=False)

Results for experiment `disease_type=all failure-type-ignored=any-error for app=best` is


Ada                     Avey                   \
                           average variance std Dev average variance std Dev   
stats_for_precision          0.291    0.052   0.227   0.192    0.028   0.167   
stats_for_recall             0.811    0.153   0.392   0.763    0.181   0.425   
stats_for_f1-score           0.428      NaN     NaN   0.307      NaN     NaN   
stats_for_f2-score           0.597      NaN     NaN   0.478      NaN     NaN   
stats_for_NDCG               0.744    0.152   0.390   0.640    0.167   0.408   
stats_for_M1                 0.649    0.228   0.477   0.500    0.250   0.500   
stats_for_M3                 0.811    0.153   0.392   0.684    0.216   0.465   
stats_for_M5                 0.811    0.153   0.392   0.763    0.181   0.425   
stats_for_position           1.267    0.329   0.573   1.655    1.054   1.026   
stats_for_length (x of gs)   3.514    1.439   1.200   4.579    0.717   0.847   

                           Avey v2                     Buoy  ... doctor_MA  \
                           average variance std Dev average  ...   average   
stats_for_precision          0.213    0.025   0.159   0.342  ...     2.074   
stats_for_recall             0.854    0.125   0.353   0.667  ...     2.649   
stats_for_f1-score           0.341      NaN     NaN   0.452  ...     2.324   
stats_for_f2-score           0.533      NaN     NaN   0.560  ...     2.508   
stats_for_NDCG               0.740    0.134   0.366   0.587  ...     2.526   
stats_for_M1                 0.610    0.238   0.488   0.487  ...     2.325   
stats_for_M3                 0.780    0.171   0.414   0.667  ...     2.649   
stats_for_M5                 0.854    0.125   0.353   0.667  ...     2.649   
stats_for_position           1.571    1.216   1.103   1.423  ...     3.374   
stats_for_length (x of gs)   4.537    0.932   0.965   2.308  ...     4.514   

                                            doctor_NJ                   \
                           variance std Dev   average variance std Dev   
stats_for_precision           0.095   0.308     0.649    0.174   0.417   
stats_for_recall              0.026   0.162     0.757    0.184   0.429   
stats_for_f1-score              NaN     NaN     0.699      NaN     NaN   
stats_for_f2-score              NaN     NaN     0.733      NaN     NaN   
stats_for_NDCG                0.047   0.218     0.747    0.183   0.428   
stats_for_M1                  0.184   0.429     0.730    0.197   0.444   
stats_for_M3                  0.026   0.162     0.757    0.184   0.429   
stats_for_M5                  0.026   0.162     0.757    0.184   0.429   
stats_for_position            0.243   0.493     1.036    0.034   0.186   
stats_for_length (x of gs)    0.722   0.850     1.297    0.371   0.609   

                           doctor_TH                  average_doctor  
                             average variance std Dev                 
stats_for_precision            0.695    0.115   0.340          0.691  
stats_for_recall               0.919    0.075   0.273          0.883  
stats_for_f1-score             0.791      NaN     NaN          0.775  
stats_for_f2-score             0.863      NaN     NaN          0.836  
stats_for_NDCG                 0.889    0.080   0.282          0.842  
stats_for_M1                   0.838    0.136   0.369          0.775  
stats_for_M3                   0.919    0.075   0.273          0.883  
stats_for_M5                   0.919    0.075   0.273          0.883  
stats_for_position             1.088    0.080   0.284          1.125  
stats_for_length (x of gs)     1.595    0.728   0.853          1.505  

[10 rows x 40 columns]

Results for experiment `disease_type=all failure-type-ignored=session-failed for app=best` is


Ada                     Avey                   \
                           average variance std Dev average variance std Dev   
stats_for_precision          0.291    0.052   0.227   0.192    0.028   0.167   
stats_for_recall             0.811    0.153   0.392   0.763    0.181   0.425   
stats_for_f1-score           0.428      NaN     NaN   0.307      NaN     NaN   
stats_for_f2-score           0.597      NaN     NaN   0.478      NaN     NaN   
stats_for_NDCG               0.744    0.152   0.390   0.640    0.167   0.408   
stats_for_M1                 0.649    0.228   0.477   0.500    0.250   0.500   
stats_for_M3                 0.811    0.153   0.392   0.684    0.216   0.465   
stats_for_M5                 0.811    0.153   0.392   0.763    0.181   0.425   
stats_for_position           1.267    0.329   0.573   1.655    1.054   1.026   
stats_for_length (x of gs)   3.514    1.439   1.200   4.579    0.717   0.847   

                           Avey v2                     Buoy  ... doctor_MA  \
                           average variance std Dev average  ...   average   
stats_for_precision          0.213    0.025   0.159   0.342  ...     2.074   
stats_for_recall             0.854    0.125   0.353   0.667  ...     2.649   
stats_for_f1-score           0.341      NaN     NaN   0.452  ...     2.324   
stats_for_f2-score           0.533      NaN     NaN   0.560  ...     2.508   
stats_for_NDCG               0.740    0.134   0.366   0.587  ...     2.526   
stats_for_M1                 0.610    0.238   0.488   0.487  ...     2.325   
stats_for_M3                 0.780    0.171   0.414   0.667  ...     2.649   
stats_for_M5                 0.854    0.125   0.353   0.667  ...     2.649   
stats_for_position           1.571    1.216   1.103   1.423  ...     3.374   
stats_for_length (x of gs)   4.537    0.932   0.965   2.308  ...     4.514   

                                            doctor_NJ                   \
                           variance std Dev   average variance std Dev   
stats_for_precision           0.095   0.308     0.649    0.174   0.417   
stats_for_recall              0.026   0.162     0.757    0.184   0.429   
stats_for_f1-score              NaN     NaN     0.699      NaN     NaN   
stats_for_f2-score              NaN     NaN     0.733      NaN     NaN   
stats_for_NDCG                0.047   0.218     0.747    0.183   0.428   
stats_for_M1                  0.184   0.429     0.730    0.197   0.444   
stats_for_M3                  0.026   0.162     0.757    0.184   0.429   
stats_for_M5                  0.026   0.162     0.757    0.184   0.429   
stats_for_position            0.243   0.493     1.036    0.034   0.186   
stats_for_length (x of gs)    0.722   0.850     1.297    0.371   0.609   

                           doctor_TH                  average_doctor  
                             average variance std Dev                 
stats_for_precision            0.695    0.115   0.340          0.691  
stats_for_recall               0.919    0.075   0.273          0.883  
stats_for_f1-score             0.791      NaN     NaN          0.775  
stats_for_f2-score             0.863      NaN     NaN          0.836  
stats_for_NDCG                 0.889    0.080   0.282          0.842  
stats_for_M1                   0.838    0.136   0.369          0.775  
stats_for_M3                   0.919    0.075   0.273          0.883  
stats_for_M5                   0.919    0.075   0.273          0.883  
stats_for_position             1.088    0.080   0.284          1.125  
stats_for_length (x of gs)     1.595    0.728   0.853          1.505  

[10 rows x 40 columns]

Results for experiment `disease_type=all failure-type-ignored=no-disease-found for app=best` is


Ada                     Avey                   \
                           average variance std Dev average variance std Dev   
stats_for_precision          0.291    0.052   0.227   0.192    0.028   0.167   
stats_for_recall             0.811    0.153   0.392   0.763    0.181   0.425   
stats_for_f1-score           0.428      NaN     NaN   0.307      NaN     NaN   
stats_for_f2-score           0.597      NaN     NaN   0.478      NaN     NaN   
stats_for_NDCG               0.744    0.152   0.390   0.640    0.167   0.408   
stats_for_M1                 0.649    0.228   0.477   0.500    0.250   0.500   
stats_for_M3                 0.811    0.153   0.392   0.684    0.216   0.465   
stats_for_M5                 0.811    0.153   0.392   0.763    0.181   0.425   
stats_for_position           1.267    0.329   0.573   1.655    1.054   1.026   
stats_for_length (x of gs)   3.514    1.439   1.200   4.579    0.717   0.847   

                           Avey v2                     Buoy  ... doctor_MA  \
                           average variance std Dev average  ...   average   
stats_for_precision          0.213    0.025   0.159   0.342  ...     2.074   
stats_for_recall             0.854    0.125   0.353   0.667  ...     2.649   
stats_for_f1-score           0.341      NaN     NaN   0.452  ...     2.324   
stats_for_f2-score           0.533      NaN     NaN   0.560  ...     2.508   
stats_for_NDCG               0.740    0.134   0.366   0.587  ...     2.526   
stats_for_M1                 0.610    0.238   0.488   0.487  ...     2.325   
stats_for_M3                 0.780    0.171   0.414   0.667  ...     2.649   
stats_for_M5                 0.854    0.125   0.353   0.667  ...     2.649   
stats_for_position           1.571    1.216   1.103   1.423  ...     3.374   
stats_for_length (x of gs)   4.537    0.932   0.965   2.308  ...     4.514   

                                            doctor_NJ                   \
                           variance std Dev   average variance std Dev   
stats_for_precision           0.095   0.308     0.649    0.174   0.417   
stats_for_recall              0.026   0.162     0.757    0.184   0.429   
stats_for_f1-score              NaN     NaN     0.699      NaN     NaN   
stats_for_f2-score              NaN     NaN     0.733      NaN     NaN   
stats_for_NDCG                0.047   0.218     0.747    0.183   0.428   
stats_for_M1                  0.184   0.429     0.730    0.197   0.444   
stats_for_M3                  0.026   0.162     0.757    0.184   0.429   
stats_for_M5                  0.026   0.162     0.757    0.184   0.429   
stats_for_position            0.243   0.493     1.036    0.034   0.186   
stats_for_length (x of gs)    0.722   0.850     1.297    0.371   0.609   

                           doctor_TH                  average_doctor  
                             average variance std Dev                 
stats_for_precision            0.695    0.115   0.340          0.691  
stats_for_recall               0.919    0.075   0.273          0.883  
stats_for_f1-score             0.791      NaN     NaN          0.775  
stats_for_f2-score             0.863      NaN     NaN          0.836  
stats_for_NDCG                 0.889    0.080   0.282          0.842  
stats_for_M1                   0.838    0.136   0.369          0.775  
stats_for_M3                   0.919    0.075   0.273          0.883  
stats_for_M5                   0.919    0.075   0.273          0.883  
stats_for_position             1.088    0.080   0.284          1.125  
stats_for_length (x of gs)     1.595    0.728   0.853          1.505  

[10 rows x 40 columns]